In [41]:
import pandas as pd
import plotly.express as px
from dash import Input, Output, dcc, html
from IPython.display import VimeoVideo
from jupyter_dash import JupyterDash
from scipy.stats.mstats import trimmed_var
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


#dcc:dash core component this contain interactive element such as slider, radio botton
#html: gives the html element like headers, sub header which is use to build layout
#jupyter dash is used to create jupyter notebook

In [14]:
# As always, we'll start by bringing our data into the project using a wrangle function.

In [42]:
#import
#task 6.4.1: Complete the wrangle function below, using the docstring as a guide. Then use your function to read the file "data/SCFP2019.csv.gz" into a DataFrame.
def wrangle(filepath):

    """Read SCF data file into ``DataFrame``.   #. this line give a summary of what the function does

    Returns only credit fearful households whose net worth is less than $2 million.    #this give detsil information about the function


    Parameters     #parameter are argument to put into our function, here we have one argument filepath we want the the data type of the path to be str
    ----------
    filepath : str
        Location of CSV file.
    """
    df=pd.read_csv(filepath)
    #create mask
    mask=(df["TURNFEAR"]==1) & (df["NETWORTH"]<2e6)
    #subset datadframe
    df=df[mask]
    
    return df

#"""""""""  doc string is a multile line string  it provide a documentation for our function  how it should be used"""""""""

In [45]:
df =wrangle("\\Users\\User\\survey of consumer finace 2022/survey of consumer finance.csv")
print(df.shape)
df.head()

(3672, 357)


,YY1,Y1,WGT,HHSEX,AGE,AGECL,EDUC,EDCL,MARRIED,KIDS,...,NWCAT,INCCAT,ASSETCAT,NINCCAT,NINC2CAT,NWPCTLECAT,INCPCTLECAT,NINCPCTLECAT,INCQRTCAT,NINCQRTCAT
20,5,51,7191.481109,2,19,1,8,2,1,0,...,1,3,1,2,1,2,5,4,2,2
21,5,52,7352.487205,2,19,1,8,2,1,0,...,1,3,1,2,1,2,5,4,2,2
22,5,53,7270.703541,2,19,1,8,2,1,0,...,1,3,1,3,1,2,5,5,2,2
23,5,54,7383.866597,2,19,1,8,2,1,0,...,1,3,1,3,1,2,5,5,2,2
24,5,55,7330.537669,2,19,1,8,2,1,0,...,1,3,1,2,1,2,5,4,2,2


In [44]:
#Build Dashboard
#It's app time! There are lots of steps to follow here, but, by the end, you'll have made an interactive dashboard! We'll start with the layout.

In [46]:
#Application Layout
#First, instantiate the application.
#Task 6.4.2: Instantiate a JupyterDash application and assign it to the variable name app.

app = JupyterDash(__name__)

C:\Users\User\anaconda3\lib\site-packages\dash\dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [ ]:
 #Task 6.4.3: Start building the layout of your app by creating a Div object that has two child objects: an H1 header that reads "Survey of Consumer Finances" and an H2 header that reads "High Variance Features".
#Note: We're going to build the layout for our application iteratively. So be prepared to return to this block of code several times as we add features.
app.layout = html.Div(
    [
       #Application title
       html.H1("Survey of Consumer Finances"),
       #Bar chart elemnts
       html.H2("High Variance Features")
 we run zooom in run it go dowm and do thesame thing.
    ]
    
)



In [ ]:
#Task 6.4.4: Add a Graph object to your application's layout. Be sure to give it the id "bar-chart".
app.layout = html.Div(
    [
       #Application title
       html.H1("Survey of Consumer Finances"),
       #Bar chart elemnts
       html.H2("High Variance Features")
        #bar chat graph
       dcc.Graph(id="bar-chart")# NO FIGURE
        
        
 we  zooom in run it go dowm and do thesame thing.
    ]
    
)




In [47]:

#this should be the final code at the ent but we start by building it gradually

app.layout = html.Div(
    [
       #Application title
       html.H1("Survey of Consumer Finances"),
       #Bar chart elemnts
       html.H2("High Variance Features"),
       #bar chat graph
       dcc.Graph( figure= serve_bar_chart(), id="bar-chart"),
       dcc.RadioItems(
           options=[
               {"label": "trimmed", "value":True},
               {"label": " not trimmed", "value":False}                              
           ],
           value=True,
           id="trim-button"
       ),
        html.H2("K-means Clustering"),
        html.H3("Number of Clusters (k)"),
        dcc.Slider(min=2, max=12, step=1, value=2, id="k-slider"),
        html.Div(id="metrics"),
        #PCA SCATTERED PLOT
        dcc.Graph(id="pca-scatter")
    ]
)    

In [ ]:

#task 6.4.5: Create a get_high_var_features function that returns the five highest-variance features in a DataFrame. Use the docstring for guidance.


In [48]:
#we use all the code below, BUSINESS LAYER
def get_high_var_features(trimmed=True, return_feat_names=True):

    """Returns the five highest-variance features of ``df``.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.

    return_feat_names : bool, default=False
        If ``True``, returns feature names as a ``list``. If ``False``
        returns ``Series``, where index is feature names and values are
        variances.
    """
    #calculate variance
    if trimmed:
        top_five_features=(
            df.apply(trimmed_var).sort_values().tail(5)    
        )
    else:
        top_five_features=df.var().sort_values().tail(5)
        # Extract names
    if return_feat_names:
        top_five_features= top_five_features.index.tolist()
            
    return top_five_features

# you can use this code to confirm if we are right , get_high_var_features(trimmed=True)

In [21]:
#Now that we have our top five features, we can use a function to return them in a bar chart.

In [22]:
# SERVICE LAYER
#Task 6.4.6: Create a serve_bar_chart function that returns a plotly express bar chart of the five highest-variance features. You should use get_high_var_features as a helper function. Follow the docstring for guidance.
def serve_bar_chart(trimmed= True):

    """Returns a horizontal bar chart of five highest-variance features.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.
    """
    #get feautures=
    top_five_features= get_high_var_features(trimmed=trimmed, return_feat_names=False)
    
    # Build bar chat
    fig=px.bar(x=top_five_features, y=top_five_features.index, orientation="h")
    fig.update_layout(xaxis_title="Variance", yaxis_title="Feature")
    
    return fig

In [ ]:
#Task 6.4.7: Use your serve_bar_chart function to add a bar chart to "bar-chart"

app.layout = html.Div(
    [
       #Application title
       html.H1("Survey of Consumer Finances"),
       #Bar chart elemnts
       html.H2("High Variance Features"),
       #bar chat graph
       dcc.Graph( figure= serve_bar_chart(), id="bar-chart") # FIGURE IS ADDED NOW
    ]
)    

# WE ZOOM OUT RUN IT THEN GO DOWN TO DO THESAME

In [ ]:
#Task 6.4.8: Add a radio button to your application's layout. It should have two options: "trimmed" (which carries the value True) and "not trimmed" (which carries the value False). Be sure to give it the id "trim-button"

app.layout = html.Div(
    [
       #Application title
       html.H1("Survey of Consumer Finances"),
       #Bar chart elemnts
       html.H2("High Variance Features"),
       #bar chat graph
       dcc.Graph( figure= serve_bar_chart(), id="bar-chart"),# FIGURE IS ADDED NOW
       dcc.RadioItems(
           options=[
               {"label": "trimmed", "value":True},
               {"label": " not trimmed", "value":False}                              
           ],
           value=True,
           id="trim-button"
       )
        
    ]
)         

# WE ZOOM OUT RUN IT THEN GO DOWN TO DO THESAME

In [ ]:
#Task 6.4.9: Add a callback decorator to your serve_bar_chart function. The callback input should be the value returned by "trim-button", and the output should be directed to "bar-chart"
app.layout = html.Div(
    [
       #Application title
       html.H1("Survey of Consumer Finances"),
       #Bar chart elemnts
       html.H2("High Variance Features"),
       #bar chat graph
       dcc.Graph( id="bar-chart"),# FIGURE IS REMOVED AGAIN SO THAT IT CAN BE DYNAMIC
       dcc.RadioItems(
           options=[
               {"label": "trimmed", "value":True},
               {"label": " not trimmed", "value":False}                              
           ],
           value=True,
           id="trim-button"
       )
        
    ]
)    

# AFTER REMOVING FIGURE WE ZOOM OUT THEN RUN IT, THEN WE STROLL TO SERVICE LAYER AND ADD CALL BACK TO 6.4.6 LIKE THIS

@app.callback(
    Output("bar-chart", "figure"), Input("trim-button","value")
)


# SERVICE LAYER
#Task 6.4.6: Create a serve_bar_chart function that returns a plotly express bar chart of the five highest-variance features. You should use get_high_var_features as a helper function. Follow the docstring for guidance.
def serve_bar_chart(trimmed= True):

    """Returns a horizontal bar chart of five highest-variance features.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.
    """
    #get feautures=
    top_five_features= get_high_var_features(trimmed=trimmed, return_feat_names=False)
    
    # Build bar chat
    fig=px.bar(x=top_five_features, y=top_five_features.index, orientation="h")
    fig.update_layout(xaxis_title="Variance", yaxis_title="Feature")
    
    return fig

#THEN# WE ZOOM OUT RUN IT THEN GO DOWN TO DO THESAME

In [ ]:
 #PRESENTATION LAYER #Task 6.4.10: Add two text objects to your application's layout: an H2 header that reads "K-means Clustering" and an H3 header that reads "Number of Clusters (k)"

app.layout = html.Div(
    [
       #Application title
       html.H1("Survey of Consumer Finances"),
       #Bar chart elemnts
       html.H2("High Variance Features"),
       #bar chat graph
       dcc.Graph( figure= serve_bar_chart(), id="bar-chart"),
       dcc.RadioItems(
           options=[
               {"label": "trimmed", "value":True},
               {"label": " not trimmed", "value":False}                              
           ],
           value=True,
           id="trim-button"
       ),
        html.H2("K-means Clustering"),
        html.H3("Number of Clusters (k)")
 )
        
    ]
)    

#THEN# WE ZOOM OUT RUN IT THEN GO DOWN TO DO THESAME

In [ ]:
Task 6.4.11: Add a slider to your application's layout. It should range from 2 to 12. Be sure to give it the id "k-slider".


app.layout = html.Div(
    [
       #Application title
       html.H1("Survey of Consumer Finances"),
       #Bar chart elemnts
       html.H2("High Variance Features"),
       #bar chat graph
       dcc.Graph( figure= serve_bar_chart(), id="bar-chart"),
       dcc.RadioItems(
           options=[
               {"label": "trimmed", "value":True},
               {"label": " not trimmed", "value":False}                              
           ],
           value=True,
           id="trim-button"
       ),
        html.H2("K-means Clustering"),
        html.H3("Number of Clusters (k)"),
        dcc.Slider(min=2, max=12, step=1, value=2, id="k-slider")
 )
        
    ]
)    

#THEN# WE ZOOM OUT RUN IT THEN GO DOWN TO DO THESAME

In [ ]:
#Task 6.4.12: Add a Div object to your applications layout. Be sure to give it the id "metrics".

app.layout = html.Div(
    [
       #Application title
       html.H1("Survey of Consumer Finances"),
       #Bar chart elemnts
       html.H2("High Variance Features"),
       #bar chat graph
       dcc.Graph( figure= serve_bar_chart(), id="bar-chart"),
       dcc.RadioItems(
           options=[
               {"label": "trimmed", "value":True},
               {"label": " not trimmed", "value":False}                              
           ],
           value=True,
           id="trim-button"
       ),
        html.H2("K-means Clustering"),
        html.H3("Number of Clusters (k)"),
        dcc.Slider(min=2, max=12, step=1, value=2, id="k-slider"),
        html.Div(id="metrics")
 )
        
    ]
)    
# we dont need to relunch our app

In [ ]:
#let work on the business layer#

#task 6.4.13: Create a get_model_metrics function that builds, trains, and evaluates KMeans model. Use the docstring for guidance. Note that, like the model you made in the last lesson, your model here should be a pipeline that includes a StandardScaler. Once you're done, submit your function to the grader.
#task 6.4.13: Create a get_model_metrics function that builds, trains, and evaluates KMeans model. Use the docstring for guidance. Note that, like the model you made in the last lesson, your model here should be a pipeline that includes a StandardScaler. Once you're done, submit your function to the grader.
def get_model_metrics (trimmed=True, k=2, return_metrics=False):

    """Build ``KMeans`` model based on five highest-variance features in ``df``.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.

    k : int, default=2
        Number of clusters.

    return_metrics : bool, default=False
        If ``False`` returns ``KMeans`` model. If ``True`` returns ``dict``
        with inertia and silhouette score.

    """
    #get high feaatures
    features= get_high_var_features(trimmed=trimmed, return_feat_names=True)
    #Create feature matrix
    X= df[features]
    # Build model
    model= make_pipeline(StandardScaler(), KMeans(n_clusters=k, random_state=42))
    model.fit(X)
    #after the above code we can use def get_model_metrics (trimmed=True, k=2, return_metrics=False) to test somethings before ew continue   
    return model 
    if return_metrics:
        i= model.named_steps["kmeans"].inertia
        ss=silhouette_score(X, model.named_steps["kmeans"].labels_)
        metrics={
            "inertia": round(i),
            "silhouette": round(ss, 3)       
        }
        return metrics
        
    return modelS
        
    

In [ ]:
#Task 6.4.14: Create a serve_metrics function. It should use your get_model_metrics to build and get the metrics for a model, and then return two objects: An H3 header with the model's inertia and another H3 header with the silhouette score.
def serve_metrics(trimmed=True, k=2):

    """Returns list of ``H3`` elements containing inertia and silhouette score
    for ``KMeans`` model.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.

    k : int, default=2
        Number of clusters.
    """
    metrics=get_model_metrics(trimmed=trimmed, k=k, return_metrics=True)
    text =[
        html.H3(f"Inertia:{metrics['inertia']}"),
        html.H3(f"Silhouette Score: {metrics['silhouette score']}")                            
    ]
    
    return text

# we zoom out and run it

In [ ]:
Task 6.4.15: Add a callback decorator to your serve_metrics function. The callback inputs should be the values returned by "trim-button" and "k-slider", and the output should be directed to "metrics"
let connect presentation to serve metric
#Task 6.4.15:ask 6.4.15: Add a callback decorator to your serve_metrics function. The callback inputs should be the values returned by "trim-button" and "k-slider", and the output should be directed to "metrics".
@app.callback(
    Output("metrics", "children"),
    input("trim-button", "value"),
    input("k-slider", "value") 
)

def serve_metrics(trimmed=True, k=2):

    """Returns list of ``H3`` elements containing inertia and silhouette score
    for ``KMeans`` model.


    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.

    k : int, default=2
        Number of clusters.
    """
    metrics=get_model_metrics(trimmed=trimmed, k=k, return_metrics=True)
    text =[
        html.H3(f"Inertia:{metrics['inertia']}"),
        html.H3(f"Silhouette Score: {metrics['silhouette score']}")                            
    ]
    
    return text
#THEN# WE ZOOM OUT RUN IT THEN GO DOWN TO DO THESAME it will update it

In [ ]:
Task 6.4.16: Add a Graph object to your application's layout. Be sure to give it the id "pca-scatter".

app.layout = html.Div(
    [
       #Application title
       html.H1("Survey of Consumer Finances"),
       #Bar chart elemnts
       html.H2("High Variance Features"),
       #bar chat graph
       dcc.Graph( figure= serve_bar_chart(), id="bar-chart"),
       dcc.RadioItems(
           options=[
               {"label": "trimmed", "value":True},
               {"label": " not trimmed", "value":False}                              
           ],
           value=True,
           id="trim-button"
       ),
        html.H2("K-means Clustering"),
        html.H3("Number of Clusters (k)"),
        dcc.Slider(min=2, max=12, step=1, value=2, id="k-slider"),
        html.Div(id="metrics"),
        #PCA SCATTERED PLOT
        dcc.Graph(id="pca-scatter")
    ]
)    

THEN# WE ZOOM OUT RUN IT THEN GO DOWN TO DO THESAME it will update it
# we have everything we need in our presentation layer

In [ ]:
# on the business layer
#Task 6.4.17: Create a function get_pca_labels that subsets a DataFrame to its five highest-variance features, reduces those features to two dimensions using PCA, and returns a new DataFrame with three columns: "PC1", "PC2", and "labels". This last column should be the labels determined by a KMeans model. Your function should you get_high_var_features and get_model_metrics as helpers. Refer to the docstring for guidance.
def get_pca_labels(trimmed= True, k=2):

    """
    ``KMeans`` labels.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.

    k : int, default=2
        Number of clusters.
    """
    #create feature matrix
    features= get_high_var_features(trimmed=trimmed, return_feat_names=True)
    X=df[features]
    # build trnsformer
    transformer= PCA(n_components=2, random_state=42)
    
    #transform data
    X_t= transformer.fit_transform(X)
    X_pca= pd.DataFrame(X_t, columns=["PC1", "PC2"])
    model= get_model_metrics(trimmed=trimmed, k=k, return_metrics= False)
    X_pca["labels"] = model.named_steps["kmeans"].labels_.astype(str)
    X_pca.sort_values("labels", inplace= True)
    
    
 #WE CAN USE THIS TO CHECK get_pca_labels().head()   
THEN# WE ZOOM OUT RUN IT THEN GO DOWN TO DO THESAME it will update it
    

In [ ]:
# LET BUILD THE SERVICES LAYER
Task 6.4.18: Create a function serve_scatter_plot that creates a 2D scatter plot of the data used to train a KMeans model, along with color-coded clusters. Use get_pca_labels as a helper. Refer to the docstring for guidance.


def serve_scatter_plot(trimmed=True, k=2):

    """Build 2D scatter plot of ``df`` with ``KMeans`` labels.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.

    k : int, default=2
        Number of clusters.
        
    """
    fig=px.scatter(
        data_frame=get_pca_labels(trimmed=trimmed, k=k),
        x="PC1",
        y="PC2",
        color="labels",
        title="PCA Representation of Clusters"
    )
    fig.update_layout(xaxis_title="PC1",yaxis_title="PC2")
    return fig

#we can use serve_scatter_plot(trimmed= True, k=5) to test it, we dont actually nned to stroll down

In [ ]:
#Task 6.4.19: Add a callback decorator to your serve_scatter_plot function. The callback inputs should be the values returned by "trim-button" and "k-slider", and the output should be directed to "pca-scatter"
#Let connect our service layer to presentation layer using call back we add call back to above


@app.callback(
    Output("pca-scatter", "figure"),
    Input("trim-button","value"),
    Input("k-slider", "value")
)

def serve_scatter_plot(trimmed=True, k=2):

    """Build 2D scatter plot of ``df`` with ``KMeans`` labels.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.

    k : int, default=2
        Number of clusters.
        
    """
    fig=px.scatter(
        data_frame=get_pca_labels(trimmed=trimmed, k=k),
        x="PC1",
        y="PC2",
        color="labels",
        title="PCA Representation of Clusters"
    )
    fig.update_layout(xaxis_title="PC1",yaxis_title="PC2")
    return fig


t#HEN# WE ZOOM OUT RUN IT THEN GO DOWN TO DO THESAME it will update it

In [ ]:
#PCA Scatter Plot
#We just made a slider that can change the inertia and silhouette scores, but not everyone will be able to understand what those changing numbers mean. Let's make a scatter plot to help them along.
#ask 6.4.16: Add a Graph object to your application's layout. Be sure to give it the id "pca-scatter".

#Just like with the bar chart, we need to get the five highest-variance features of the data, so let's start with that.


In [ ]:
#now we can use those five features to make the actual scatter plot.
#Task 6.4.18: Create a function serve_scatter_plot that creates a 2D scatter plot of the data used to train a KMeans model, along with color-coded clusters. Use get_pca_labels as a helper. Refer to the docstring for guidance.





1 first we do the following steps
Div is like a section of our website
inside div we have our element (tile for our entire application and the subsection title)
H1: HEADER 1 BIGGEST HEADER
H2: BAR CHART HEADER
DCC: Dash core component module which is a place holder for the graph
id: is use to locate it as we pass information between our layers

after these step we go down and run it

note:we just created a place holder in our presentation layer now we want to create function in our business layer


2)we buid the business layer, we will not run it
 evrything in task 6.4.5 ( note, you can use this code to confirm if we are right , get_high_var_features())
    
after this we buid the service layer  FOR CONNECTION
   
3)service bar chat 
evrything in task 6.4.6 ( ( note, you can use this code to confirm if we are right , serve_bar_chart(trimmed= False))

then we impute this into our presentation layer (Task 6.4.7: Use your serve_bar_chart function to add a bar chart to "bar-chart".) which is already in our presentation layer. 
                    we zoom out gp down and run it
                         
                         
                         
                         
we go back to the presentation layer and do this
#Task 6.4.8: Add a radio button to your application's layout. It should have two options: "trimmed" (which carries the value True) and "not trimmed" (which carries the value False). Be sure to give it the id "trim-button".
 run it then go down to run it 
                         
                         
                         
Now that we have code to create our bar chart, a place in our app to put it, and a button to manipulate it, let's connect all three elements. 
  
                         
then, we go back to the presentation and remove the  figure= serve_bar_chart(), we run it, then go back to 6.49 to add our call back. callback  is what we can do to change the dynamicity of our presentation, it is a way of connecting  our serve bar function in serevice  layer to the  graph and radio button in our presentation. decorator piece is something we add before any function in python given the function power.                      
                         
Task 6.4.9: Add a callback decorator to your serve_bar_chart function. The callback input should be the value returned by "trim-button", and the output should be directed to "bar-chart". we go down and run to see   
                         
next we go to the slider and build it (K-means Slider and Metrics Okay, so now our app has a radio button, but that's only one thing for a viewer to interact with. Buttons are fun, but what if we made a slider to help people see what it means for the number of clusters to change. Let's do it!Again, start by adding some objects to the layout         
 we go to the presentation and add  
H2: k means cluster
 H3; NUMBERS OF CLUSTERS Task 6.4.10: Add two text objects to your application's layout: an H2 header that reads "K-means Clustering" and an H3 header that reads "Number of Clusters (k)".
 we run it  TO MEMORY and go down to run it THE APP
                                       
                                       
WE go up again
Task 6.4.11: Add a slider to your application's layout. It should range from 2 to 12. Be sure to give it the id "k-slider".
we use dcc. slider(min.......)
we run to memory then go down to app amd run  
                                       
                                       
we add place holder
html. div(,,,,)#task 6.4.12: Add a Div object to your applications layout. Be sure to give it the id "metrics".
#So now we have a bar chart that changes with a radio button, and a slider that changes... well, nothing yet. Let's give it a model to work with.      
we dont run anything just leave it and go to next step. 
                                       
now let work on the business layer, we wiil be working on a function called get model metric
 (Task 6.4.13: Create a get_model_metrics function that builds, trains, and evaluates KMeans model. Use the docstring for guidance. Note that, like the model you made in the last lesson, your model here should be a pipeline that includes a StandardScaler. Once you're done, submit your function to the grader.                                      
   
we run it,we can use this to confirm our code ...get_model_metrics(trimmed=True, k=5, return_metrics=True)     
  

  
next  
  
we do all the code in 6.4.14 Task 6.4.14: Create a serve_metrics function. It should use your get_model_metrics to build and get the metrics for a model, and then return two objects: An H3 header with the model's inertia and another H3 header with the silhouette score.  
we use this to check it. serve_metrics(trimmed=True, k=2)
we run it
  
next 
we add callback to connect our presentation layer to our serve metric layer to our 6.4.15 then we run it... we dont need to create another one we just adjust the 6.4.14
we add input output input. 
once this we run it to memory then we go downn to run it is done we gO
  

NEXT
we add pca scattered plot to our grah
  i.e dicc. and graph
we zoom out run it go down and run it
  
next the business layer
  write code in Task 6.4.17
  we can use(get_pca_labels().head() to get pca 
we run it
             
             
next 
we create a serve scattered plot in by using the code in 6.4.18
 
next let do the service layer
by adding call back  we work with 6.4.18 by adding call back and run it  making it 6.4.19 we dont need to create another one we just add call back
             
we zoom out run it go down and run it this should lunch our new application with everything working
             

            
WHAT WE LEARN.....          
             
1)we import our data
2)we learn doocstring what they are how they are formtted and how dey help others use our code easily
3)we build web application adding html and dash core component
etc
4)we have three element bar chat, slider and scattered plot, 
we organized them using the presentation layer (visualization, interactive element) what the user sees), the business layer(
( which access our work and ) the services layer  which is the middle man

The presentation layer: what the user sees (display), its include the visualization, interactive element
The business layer: which do our work, for Calculting variance, train our model and performing pca
The services layer:  which is the middle man that connect the presentation to service layer. it translact information from thepresentation layr to a form the business layer understand vice versa